#### Ciencia de datos - Primavera 2024
##### Integrantes: Magdalena Cobb, Pedro García Vassallo, Marcos Olavarría
### Trabajo Práctico 4

# 1. LECTURA

In [1]:
#Paquetes y librerías a usar
#!pip install openpyxl
#!pip install statsmodels

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
#from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [73]:
#Lectura del documento
indi_2024 = pd.read_excel('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/usu_individual_T124.xlsx') 
indi_2004 = pd.read_stata('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/Individual_t104.dta')
hoga_2024 = pd.read_excel('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/usu_hogar_T124.xlsx')
hoga_2004 = pd.read_stata('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/Hogar_t104.dta')

# Maggie: C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/
# Peter: 'C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 4/
# Marck: C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 4/


# 2. Selección de GBA y merge

a. Eliminen todas las observaciones que no corresponden a los
aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos
Aires, y unan ambos trimestres en una sola base.

In [195]:
#Filtramos las bases para tener solo Buenos Aires
ind_2024 = indi_2024[(indi_2024['AGLOMERADO'] == 33) | (indi_2024['AGLOMERADO'] == 32)]
hog_2024 = hoga_2024[(hoga_2024['AGLOMERADO'] == 33) | (hoga_2024 ['AGLOMERADO'] == 32 )]

ind_2004 = indi_2004[(indi_2004['aglomerado'] == 'Ciudad de Buenos Aires') | (indi_2004 ['aglomerado'] == 'Partidos del GBA' )]
hog_2004 = hoga_2004[(hoga_2004['aglomerado'] == 'Ciudad de Buenos Aires') | (hoga_2004 ['aglomerado'] == 'Partidos del GBA' )]

ind_2004.columns = ind_2004.columns.str.upper()
hog_2004.columns = hog_2004.columns.str.upper()


In [194]:
#Eliminación de columnas redundantes (preguntas hechas solo para ocupados o desocupados)
#tambien sacamos las que son del interior
redundantes_ind04 = [
    "CAT_INAC",
    "PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3",
    "PP11A", "PP11B_COD", "PP11B1", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA", "PP11C", "PP11C99", "PP11D_COD",
    "PP11G_ANO", "PP11G_MES", "PP11G_DIA", "PP11L", "PP11L1", "PP11M", "PP11N", "PP11O", "PP11P", "PP11Q", "PP11R", 
    "PP11S", "PP11T", "PP02C1", "PP02C2", "PP02C3", "PP02C4", "PP02C5", "PP02C6", "PP02C7", "PP02C8", "PP02E", 
    "PP02H", "PP02I", "CAT_OCUP", "PP03C", "PP03D", "PP3E_TOT", "PP3F_TOT", "PP03G", "PP03H", "PP03I", "PP03J", 
    "INTENSI", "PP04A", "PP04B_COD", "PP04B1", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", "PP04C", 
    "PP04C99", "PP04D_COD", "PP04G", "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP05C_1", "PP05C_2", "PP05C_3", 
    "PP05E", "PP05F", "PP05H", "PP06A", "PP06C", "PP06E", "PP06H", "PP06D", "PP07A", "PP07C", "PP07D", "PP07E", "PP07F1", "PP07F2", 
    "PP07F3", "PP07F4", "PP07F5", "PP07G1", "PP07G2", "PP07G3", "PP07G4", "PP07G_59", "PP07H", "PP07I", "PP07J", 
    "PP07K", "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR", "TOT_P12", "PJ1_1", 
    "PJ2_1", "PJ3_1", "IDIMPP", 'PP09A', 'PP09A_ESP',  'PP09B', 'PP09C',  'PP09C_ESP',  'PP10A','PP10C',  'PP10D', 'PP10E',  'IDECINDR', 'PDECINDR', 'MAS_500']

redundantes_hog04 = [
    "II3",
    "II3_1",
    "II6",
    "II6_1", 
    'IDIMPH', 'IDECIFR','PDECIFR', 'IDECCFR', 'PDECCFR'
]

redundantes_ind24 = [
    "PP02C1", "PP02C2", "PP02C3", "PP02C4", "PP02C5", "PP02C6", "PP02C7", "PP02C8",
    "PP02E", "PP02H", "PP02I", "PP10A", "PP10C", "PP10D", "PP10E", "PP11A",
    "PP11B_COD", "PP11B1", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA", "PP11C", 
    "PP11C99", "PP11D_COD", "PP11G_ANO", "PP11G_MES", "PP11G_DIA", "PP11L", "PP11L1",
    "PP11M", "PP11N", "PP11O", "PP11P", "PP11Q", "PP11R", "PP11S", "PP11T",
    "PP03C", "PP03D", "PP3E_TOT", "PP3F_TOT", "PP03G", "PP03H", "PP03I", "PP03J",
    "INTENSI", "PP04A", "PP04B_COD", "PP04B1", "PP04B2", "PP04B3_MES", "PP04B3_ANO",
    "PP04C", "PP04C99", "PP04D_COD", "PP04G", "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", 'PP04B3_DIA',
    "PP05C_1", "PP05C_2", "PP05C_3", "PP05E", "PP05F", "PP05H", "PP06A", "PP06C",
    "PP06D", "PP06E", "PP06H", "PP07A", "PP07C", "PP07D", "PP07E", "PP07F1", "PP07F2",
    "PP07F3", "PP07F4", "PP07F5", "PP07G1", "PP07G2", "PP07G3", "PP07G4", "PP07G_59",
    "PP07H", "PP07I", "PP07J", "PP07K", "PP08D1", "PP08D4", "PP08F1", "PP08F2", 
    "PP08J1", "PP08J2", "PP08J3", "PP09A", "PP09A_ESP", "PP09B", "PP09C", "PP09C_ESP",
    "P21", "DECOCUR", "IDECOCUR", "RDECOCUR", "GDECOCUR", "PDECOCUR", "ADECOCUR",
    "PONDIIO", "TOT_P12", "P47T", "DECINDR", "IDECINDR", "RDECINDR", "GDECINDR", 
    "PDECINDR", "ADECINDR", "PONDII",  'PP10A',  'PP10C', 'PP10D',  'PP10E', 'CH15_COD', 'CH16_COD', 'IDECINDR', 'PDECINDR', 'MAS_500'
]

redundantes_hog24 = [
    "II3",
    "II3_1",
    "II6",
    "II6_1", 'IV1_ESP', 'IV3_ESP', 'IV7_ESP', 'II7_ESP', 'II8_ESP' , 'IDECIFR','PDECIFR', 'IDECCFR', 'PDECCFR'

]



In [196]:
#Sacamos las variables redundantes
ind_2004 = ind_2004.drop(columns = redundantes_ind04)
ind_2024 = ind_2024.drop(columns = redundantes_ind24)
hog_2004 = hog_2004.drop(columns = redundantes_hog04)
hog_2024 = hog_2024.drop(columns = redundantes_hog24)

In [198]:
#Sacamos de las bases individuales variables que van a aparecer en hogar, para emprolijar el merge
lista_con = ['ANO4', 'TRIMESTRE', 'REGION', 'AGLOMERADO', 'PONDERA', 'ITF', 'DECIFR', 'IDECIFR', 
             'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'RDECCFR', 'GDECCFR', 'ADECCFR']

lista_con2 = ['ANO4', 'TRIMESTRE', 'REGION',  'AGLOMERADO', 'PONDERA', 'ITF', 'DECIFR', 'IDECIFR', 'RDECIFR',
              'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'RDECCFR', 'GDECCFR', 'ADECCFR','PONDIH']

ind_2004 = ind_2004.drop (columns = (lista_con))
ind_2024 = ind_2024.drop(columns = (lista_con2))

In [199]:
#Mergeamos bases hogar + individual para cada año
combined_04 = pd.merge(ind_2004, hog_2004, on=['CODUSU', 'NRO_HOGAR'], how='outer') #7647 obs x 124 variables
combined_24 = pd.merge(ind_2024,hog_2024,on=["CODUSU","NRO_HOGAR"], how = 'outer') #7051 obs x 120 variables


# 2. Limpieza
Limpien la base de datos tomando criterios que hagan sentido. Explicar
cualquier decisión como el tratamiento de valores faltantes (missing
values), extremos (outliers), o variables categóricas. Justifique sus
decisiones.

In [73]:
# Se usa base de datos:
# 2024: 
# - combined_24

# 2004:
# - combined_04

##### Limpieza de datos

In [428]:
# Limpieza de datos tomando criterios que hagan sentido
# print(combined_24.describe())


In [429]:
# # Verificar si cada columna tiene algún valor negativo 2024
# negativos = (combined_24.select_dtypes(include=[float, int]) < 0).any()

# # Filtrar las columnas que contienen valores negativos (solo mostrar las que son True)
# columnas_con_negativos = negativos[negativos]  # Esto te devuelve solo las columnas que son True

# print(columnas_con_negativos)


In [200]:
#Eliminamos valores negativos de la base 2024
# Verificar si cada columna tiene algún valor negativo
# negativos = (combined_24.select_dtypes(include=[float, int]) < 0).any()

# Filtrar las filas que contienen valores negativos (eliminar las filas con cualquier valor negativo)
combined_24_clean = combined_24[~(combined_24.select_dtypes(include=[float, int]) < 0).any(axis=1)]

# # Mostrar el DataFrame limpio
# print(combined_24_clean)

combined_24 = combined_24_clean
combined_24 #elimina 397 observaciones


,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,RDECCFR,GDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
1,TQRMNOPPQHJNOPCDEIJAH00802435,1,2,1,2,2,1971-08-17 00:00:00,52,2,1,...,12,12.0,12,0,2,0,98,0,0,0
2,TQRMNOPPQHJNOPCDEIJAH00802435,1,4,1,3,1,2002-01-04 00:00:00,22,5,1,...,12,12.0,12,0,2,0,98,0,0,0
3,TQRMNOPPQHKKMMCDEIIAD00858017,1,1,1,1,2,1971-05-09 00:00:00,52,5,2,...,12,12.0,12,0,1,0,98,0,0,0
4,TQRMNOPPQHKMQUCDEIJAH00861670,1,1,1,1,2,1948-07-24 00:00:00,75,4,1,...,8,8.0,8,3263,1,0,98,0,0,0
5,TQRMNOPPRHJNOPCDEIJAH00793617,1,1,1,1,2,1982-09-28 00:00:00,41,1,4,...,4,4.0,4,3206,1,0,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7046,TQUMNOQSVHMOTSCDEIJAH00859221,1,1,1,1,2,1962-08-07 00:00:00,61,2,1,...,8,8.0,9,1834,1,0,98,0,0,0
7047,TQUMNOQSVHMOTSCDEIJAH00859221,1,2,1,2,1,1971-01-13 00:00:00,53,2,1,...,8,8.0,9,1834,1,0,98,0,0,0
7048,TQVMNORVXHLOTOCDEIJAH00855319,1,1,1,1,2,1988-08-22 00:00:00,35,5,1,...,12,12.0,12,0,1,0,2,0,0,0
7049,TQVMNORVXHLOTOCDEIJAH00855319,1,2,1,3,2,2006-06-26 00:00:00,17,5,1,...,12,12.0,12,0,1,0,2,0,0,0


In [201]:
#Limpiar negativos 2004
# Verificar si cada columna tiene algún valor negativo 2004 
# negativos = (combined_04.select_dtypes(include=[float, int]) < 0).any()

# Filtrar las filas que contienen valores negativos (eliminar las filas con cualquier valor negativo)
combined_04_clean = combined_04[~(combined_04.select_dtypes(include=[float, int]) < 0).any(axis=1)]

# # Mostrar el DataFrame limpio
# print(combined_04_clean)

combined_04 = combined_04_clean
combined_04 #elimina 21 observaciones


,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH06,CH07,CH08,CH09,...,DECCFR,RDECCFR,GDECCFR,ADECCFR,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,125047,1.0,1.0,Sí,Jefe,Varón,80.0,Casado,Obra social (incluye PAMI),Sí,...,10,10,10,10,2.0,0.0,98.0,0.0,0.0,0.0
1,125047,1.0,2.0,Sí,Cónyuge/Pareja,Mujer,75.0,Casado,Obra social (incluye PAMI),Sí,...,10,10,10,10,2.0,0.0,98.0,0.0,0.0,0.0
2,125098,1.0,1.0,Sí,Jefe,Varón,79.0,Viudo,Obra social (incluye PAMI),Sí,...,07,06,06,04,1.0,0.0,98.0,0.0,0.0,0.0
3,125169,1.0,1.0,Sí,Jefe,Varón,72.0,Unido,No paga ni le descuentan,Sí,...,01,01,01,01,2.0,0.0,3.0,0.0,0.0,0.0
4,125169,1.0,2.0,Sí,Cónyuge/Pareja,Mujer,66.0,Unido,No paga ni le descuentan,Sí,...,01,01,01,01,2.0,0.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7642,288171,1.0,3.0,0.0,Hijo/Hijastro,Mujer,1.0,Soltero,Mutual/Prepaga/Servicio de emergencia,Menor de 2 años,...,08,08,08,09,Servicio doméstico,0.0,2.0,0.0,0.0,0.0
7643,288405,1.0,1.0,Sí,Jefe,Varón,51.0,Casado,Obra social (incluye PAMI),Sí,...,02,02,02,02,2.0,0.0,98.0,0.0,0.0,0.0
7644,288405,1.0,2.0,Sí,Cónyuge/Pareja,Mujer,55.0,Casado,Obra social (incluye PAMI),Sí,...,02,02,02,02,2.0,0.0,98.0,0.0,0.0,0.0
7645,288405,1.0,3.0,Sí,Hijo/Hijastro,Mujer,12.0,Soltero,Obra social (incluye PAMI),Sí,...,02,02,02,02,2.0,0.0,98.0,0.0,0.0,0.0


#### Valores Faltantes
- 2004 no tiene
- 2024 se eliminaron los más brutos y se imputaron por la mediana los menos

##### 2004
-Tenia una columna con valores Nan = IX_MEN10, es el 50%, se imputa por la mediana.


In [202]:
# # Verificar si cada columna tiene algún valor NaN 2004
nan_values04 = combined_04.isna().any()
columnas_con_nan04 = nan_values04[nan_values04]  # Esto te devuelve solo las columnas que tienen NaN
columnas_con_nan04 = columnas_con_nan04.index
print(columnas_con_nan04)

Index(['IX_MEN10'], dtype='object')


In [ ]:
# proporcion_columna04 = combined_04['IX_MEN10'].isnull().sum() / len(combined_04) * 100
# proporcion_columna04 #El 50% de la columna es nan. puedo imputar por la mediana. 

# #Se imputa por la mediana
# combined_04[columnas_con_nan04] = combined_04[columnas_con_nan04].fillna(combined_04[columnas_con_nan04].median())

In [203]:
combined_04 = combined_04.drop(columns = (columnas_con_nan04))

##### 2024
- Tiene columnas con valores Nan
- Se revisa la proporción de Nan que hay por columna
- Elimino las columnas que tienen proporción de Nans > 90
- Imputo por la mediana en las columnas que su proporción de Nans < 90

In [204]:
# # Verificar si cada columna tiene algún valor NaN 2024
nan_values24 = combined_24.isna().any()

# # Filtrar las columnas que contienen valores NaN (solo mostrar las que son True)
columnas_con_nan24 = nan_values24[nan_values24] # Esto te devuelve solo las columnas que tienen NaN

print(columnas_con_nan24)

columnas_con_nan24 = columnas_con_nan24.index


CH14       True
IMPUTA     True
IDECCFR    True
PDECCFR    True
dtype: bool


In [206]:
# # Obtengo las columnas que contienen datos faltantes, reviso que proporción de los valores faltas

# # Porcentaje de valores faltantes por columna
lista_nans_poco = []
lista_nans_50 = []
lista_nans_60 = []
lista_nans_70 = []
lista_nans_80 = []
lista_nans_90 = []
lista_nans_100 = []

for i in columnas_con_nan24:
    proporcion_columna24 = combined_24[i].isnull().sum() / len(combined_24) * 100
    
    if proporcion_columna24 < 50:
        lista_nans_poco.append(i)

    elif proporcion_columna24 >= 50 and proporcion_columna24 < 60:
        lista_nans_50.append(i)

    elif proporcion_columna24 >= 60 and proporcion_columna24 < 70:
        lista_nans_60.append(i)

    elif proporcion_columna24 >= 70 and proporcion_columna24 < 80:
        lista_nans_70.append(i)

    elif proporcion_columna24 >= 80 and proporcion_columna24 < 90:
        lista_nans_80.append(i)

    elif proporcion_columna24 >= 90 and proporcion_columna24 < 100:
        lista_nans_90.append(i)

    elif proporcion_columna24 == 100:
        lista_nans_100.append(i)
    

In [207]:
# Elimino las variables que tienen más de 90% de datos Nan. Me fijo antes cuantas columnas son
columnas_elim = lista_nans_90 + lista_nans_100
combined_24 = combined_24.drop(columns = (columnas_elim)) #se eliminaron 6 columnas

In [208]:
# Imputo por la mediana en las columnas que su proporción de Nans < 90
columnas_ok = lista_nans_poco + lista_nans_50 +lista_nans_60 + lista_nans_70 + lista_nans_80
combined_24[columnas_ok] = combined_24[columnas_ok].fillna(combined_24[columnas_ok].median())


In [209]:
#AHORA VEMOS QUE NO HAY MAS NANS EN LA DE 2024
nan_values24 = combined_24.isna().any()

# # Filtrar las columnas que contienen valores NaN (solo mostrar las que son True)
columnas_con_nan24 = nan_values24[nan_values24] # Esto te devuelve solo las columnas que tienen NaN

print(columnas_con_nan24)

columnas_con_nan24 = columnas_con_nan24.index


Series([], dtype: bool)


##### Outliers 

In [ ]:
numerica04 = [ 'P47T', 'DECINDR', 
                   'RDECINDR', 'GDECINDR', 'ADECINDR', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M',
                    'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI', 'ANO4',
                       'PONDERA', 'IV1_ESP', 'IV3_ESP', 'IV7_ESP',
                        'II5_1', 'II7_ESP', 'II8_ESP', 'IX_TOT', 'IX_MAYEQ10', 'ITF', 'DECIFR', 
                        'RDECIFR', 'GDECIFR',  'ADECIFR', 'IPCF', 'DECCFR', 'RDECCFR',
                          'GDECCFR', 'ADECCFR']


#es igual que no categorica pero sin codusu, porque no tiene sentido sacar outliers de ahi. 
#estoy sacando COMPONENTE , CH06, CH04, ESTADO xq son tipo de dato category 
# 'REALIZADA', 'TRIMESTRE', 'REGION', 'AGLOMERADO', 'IV2', 'II1', 'II2',
      # 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4', 'H15', 'V1',
      # 'V2', 'V21', 'V22', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
      # 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A',
      # 'V19_B', 'II4_1', 'IV8', 'IV12_2', 'IV12_3', 'IV12_1', 'II4_2', 'II4_3',
      # 'CH13', 'II5', 'IV5', 'REALIZADA', 'TRIMESTRE', 'AGLOMERADO', 'REGION', 'II1','II2', 'IV2'

# y saque NRO HOGAR

#es igual que no categorica pero sin codusu, porque no tiene sentido sacar outliers de ahi. 

In [213]:
# Verifica el tipo de datos de todas las columnas en el DataFrame
print(combined_04[numerica04].dtypes)

# Filtrar las columnas que son de tipo 'category'
categorical_columns = combined_04[numerica04].select_dtypes(include=['category']).columns

# Mostrar las columnas que son de tipo 'category'
print(f"Columnas de tipo 'category': \n{categorical_columns}")

CH15_COD       object
CH16_COD       object
P47T          float64
DECINDR        object
RDECINDR       object
GDECINDR       object
ADECINDR       object
V2_M          float64
V3_M          float64
V4_M          float64
V5_M          float64
V8_M          float64
V9_M          float64
V10_M         float64
V11_M         float64
V12_M         float64
V18_M         float64
V19_AM        float64
V21_M         float64
T_VI          float64
ANO4          float64
PONDERA       float64
IV1_ESP        object
IV3_ESP        object
IV7_ESP        object
II5_1         float64
II7_ESP        object
II8_ESP        object
IX_TOT        float64
IX_MAYEQ10    float64
ITF           float64
DECIFR         object
RDECIFR        object
GDECIFR        object
ADECIFR        object
IPCF          float64
DECCFR         object
RDECCFR        object
GDECCFR        object
ADECCFR        object
dtype: object
Columnas de tipo 'category': 
Index([], dtype='object')


In [214]:
# Outliers
# COMBINED 04

combined_04[numerica04] = combined_04[numerica04].apply(pd.to_numeric, errors='coerce')

# Calculamos los cuartiles y el IQR solo para las columnas numéricas
Q1 = combined_04[numerica04].quantile(0.25)
Q3 = combined_04[numerica04].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Detectar outliers (donde los valores están fuera de los límites)
outliers = (combined_04[numerica04] < lower_limit) | (combined_04[numerica04] > upper_limit)

# Mostrar una vista de los outliers por fila
outliers_data = combined_04[outliers.any(axis=1)]

# Opcional: Mostrar el total de outliers por columna
outliers_count = outliers.sum()
print(f"Total de outliers por columna: \n{outliers_count}")

# Mostrar las filas que contienen outliers
print(f"Filas con outliers: \n{outliers_data}")

# Eliminar filas con outliers
combined_04_no_outliers = combined_04[~outliers.any(axis=1)]
print(f"Datos después de eliminar outliers: \n{combined_04_no_outliers.head()}")

combined_04 = combined_04_no_outliers

Total de outliers por columna: 
CH15_COD         5
CH16_COD         0
P47T           496
DECINDR          0
RDECINDR         0
GDECINDR         0
ADECINDR         0
V2_M           764
V3_M             8
V4_M            13
V5_M           107
V8_M            71
V9_M             8
V10_M           15
V11_M           13
V12_M          229
V18_M            5
V19_AM           0
V21_M          107
T_VI          1118
ANO4             0
PONDERA        363
IV1_ESP          0
IV3_ESP          0
IV7_ESP          0
II5_1           76
II7_ESP          0
II8_ESP          0
IX_TOT         324
IX_MAYEQ10     194
ITF            510
DECIFR           0
RDECIFR          0
GDECIFR          0
ADECIFR          0
IPCF           550
DECCFR           0
RDECCFR          0
GDECCFR          0
ADECCFR          0
dtype: int64
Filas con outliers: 
        CODUSU  NRO_HOGAR COMPONENTE H15            CH03   CH04  CH06  \
0     125047          1.0        1.0  Sí            Jefe  Varón  80.0   
1     125047          1.0   

##### 2024

In [215]:
#Lista de todas las variables de la base 2004
lista04 = combined_04.columns.to_list()
len(lista04)

117

In [ ]:
#Para la base 2004, reemplazo los valores de texto de las variables binarias a 0 y 1 para ya usarlas como dummies. 
#CREO QUE ESTO NO SIRVEEE
reemplazosbinarias = {
    "Sí" : 1, 
    "No" : 0, 
    "S" : 1, 
    "Varón" : 1, 
    "Mujer" : 0
}

combined_04 = combined_04.applymap(lambda x: reemplazosbinarias.get(x, x))
combined_24 = combined_24.applymap(lambda x: reemplazosbinarias.get(x, x))

C:\Users\magda\AppData\Local\Temp\ipykernel_2880\1164433441.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_04 = combined_04.applymap(lambda x: reemplazosbinarias.get(x, x))
C:\Users\magda\AppData\Local\Temp\ipykernel_2880\1164433441.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_24 = combined_24.applymap(lambda x: reemplazosbinarias.get(x, x))


In [222]:
#variables numericas para sacar outliers
numerica24 = ['NRO_HOGAR', 'COMPONENTE', 'CH05', 'CH06', 'ESTADO', 'V2_M', 
                  'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM',
                    'V21_M', 'T_VI', 'ANO4', 'TRIMESTRE', 'REALIZADA', 'REGION', 'MAS_500', 'AGLOMERADO',
                      'PONDERA', 'II1', 'II2', 'II5_1',
                        'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'ITF', 'DECIFR', 'GDECIFR', 'ADECIFR',
                          'IPCF', 'DECCFR', 'RDECCFR', 'GDECCFR', 'ADECCFR', 'PONDIH', 'VII1_1', 'VII1_2', 
                          'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4']

In [ ]:
# # COMBINED 24
# import pandas as pd

# # Supongamos que tienes un DataFrame llamado combined_04
# # Primero seleccionamos solo las columnas numéricas para el análisis
# numerical_columns = combined_24.select_dtypes(include=['float64', 'int64']).columns

# # Calculamos los cuartiles y el IQR solo para las columnas numéricas
# Q1 = combined_24[numerical_columns].quantile(0.25)
# Q3 = combined_24[numerical_columns].quantile(0.75)
# IQR = Q3 - Q1

# # Definir los límites inferior y superior
# lower_limit = Q1 - 1.5 * IQR
# upper_limit = Q3 + 1.5 * IQR

# # Detectar outliers (donde los valores están fuera de los límites)
# outliers = (combined_24[numerical_columns] < lower_limit) | (combined_24[numerical_columns] > upper_limit)

# # Mostrar una vista de los outliers por fila
# outliers_data = combined_24[outliers.any(axis=1)]

# # Opcional: Mostrar el total de outliers por columna
# outliers_count = outliers.sum()
# print(f"Total de outliers por columna: \n{outliers_count}")

# # Mostrar las filas que contienen outliers
# print(f"Filas con outliers: \n{outliers_data}")

# # Eliminar filas con outliers
# #combined_04_no_outliers = combined_04[~outliers.any(axis=1)]
# #print(f"Datos después de eliminar outliers: \n{combined_04_no_outliers.head()}")

In [223]:
#outliers 2024
combined_24[numerica24] = combined_24[numerica24].apply(pd.to_numeric, errors='coerce')

# Calculamos los cuartiles y el IQR solo para las columnas numéricas
Q1 = combined_24[numerica24].quantile(0.25)
Q3 = combined_24[numerica24].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Detectar outliers (donde los valores están fuera de los límites)
outliers24 = (combined_24[numerica24] < lower_limit) | (combined_24[numerica24] > upper_limit)

# Mostrar una vista de los outliers por fila
outliers_data24 = combined_24[outliers24.any(axis=1)]

# Opcional: Mostrar el total de outliers por columna
outliers_count24 = outliers24.sum()
print(f"Total de outliers por columna: \n{outliers_count24}")

# Mostrar las filas que contienen outliers
print(f"Filas con outliers: \n{outliers_data24}")

# Eliminar filas con outliers
combined_24_no_outliers = combined_24[~outliers24.any(axis=1)]
print(f"Datos después de eliminar outliers: \n{combined_24_no_outliers.head()}")

combined_04 = combined_04_no_outliers

Total de outliers por columna: 
NRO_HOGAR       55
COMPONENTE     141
CH05             0
CH06             0
ESTADO           0
V2_M          1112
V3_M            11
V4_M             3
V5_M           260
V8_M            70
V9_M             1
V10_M           56
V11_M           68
V12_M          155
V18_M            7
V19_AM           0
V21_M          438
T_VI          1603
ANO4             0
TRIMESTRE        0
REALIZADA        0
REGION           0
MAS_500          0
AGLOMERADO    1310
PONDERA        161
II1             23
II2              0
II5_1           62
IX_TOT         128
IX_MEN10       308
IX_MAYEQ10      70
ITF            382
DECIFR           0
GDECIFR          0
ADECIFR          0
IPCF           457
DECCFR           0
RDECCFR          0
GDECCFR          0
ADECCFR          0
PONDIH         110
VII1_1         274
VII1_2         369
VII2_1           0
VII2_2         829
VII2_3         304
VII2_4          42
dtype: int64
Filas con outliers: 
                             CODUSU  NRO_

#### Variables categoricas

##### 2004

In [224]:
#Armo una lista con todas las variables que no quiero que sean dummies. Incluye numercias continuas y binarias (0,1).
nocategorica04 = ['CODUSU',
 'NRO_HOGAR',
 'COMPONENTE',
 'CH06',
 'ESTADO', #Estado es categorica, pero lo dejo porque despues vamos a usar una dummy de desocupado directamente. 
 'P47T',
 'DECINDR',
 'IDECINDR',
 'RDECINDR',
 'GDECINDR',
 'PDECINDR',
 'ADECINDR',
 'V2_M',
 'V3_M',
 'V4_M',
 'V5_M',
 'V8_M',
 'V9_M',
 'V10_M',
 'V11_M',
 'V12_M',
 'V18_M',
 'V19_AM',
 'V21_M',
 'T_VI',
 'REALIZADA',
 'ANO4',
 'TRIMESTRE',
 'REGION',
 'MAS_500',
 'AGLOMERADO',
 'PONDERA',
 'IV2',
 'II1',
 'II2',
 'II5_1',
 'IX_TOT',
 'IX_MEN10',
 'IX_MAYEQ10',
 'ITF',
 'DECIFR',
 'RDECIFR',
 'GDECIFR',
 'ADECIFR',
 'IPCF',
 'DECCFR',
 'RDECCFR',
 'GDECCFR',
 'ADECCFR',
 'VII1_1',
 'VII1_2',
 'VII2_1',
 'VII2_2',
 'VII2_3',
 'VII2_4', 
'H15', #a partir de aca son binarias. 1/0
 'V1',
 'V2',
 'V21',
 'V22',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19_A',
 'V19_B', 
 'II4_1', 
 'IV8',
 'IV12_2',
 'IV12_3', 'IV12_1', 'II4_2', 'II4_3', 'CH04', 'CH13', 'II5', 'IV5']

len(nocategorica04)

89

In [225]:
#Armo una lista de las variables categoricas para hacer dummies. Son todas las variables menos las binarias y las numericas continuas. 
categorica04 = list(set(lista04) - set(nocategorica04))
len(categorica04)

31

In [226]:
dummies04 = pd.get_dummies(combined_04[categorica04])
combined_04_cat = pd.concat([combined_04, dummies04], axis = 1)

### combined_04_cat ----- incluye dummies Y las variables originales. 

combined_04 = combined_04_cat.drop(columns = categorica04)

##### 2024

In [227]:
#Lista de todas las variables de la base 2024
lista24 = combined_24.columns.to_list()
len(lista24)

108

In [228]:
#misma lista para 2024. variables numericas continuas (que no quiero convertir en dummies)
nocategorica24 = ['CODUSU',
 'NRO_HOGAR',
 'COMPONENTE',
 'CH05',
 'CH06',
 'ESTADO',
 'V2_M',
 'V3_M',
 'V4_M',
 'V5_M',
 'V8_M',
 'V9_M',
 'V10_M',
 'V11_M',
 'V12_M',
 'V18_M',
 'V19_AM',
 'V21_M',
 'T_VI',
 'ANO4',
 'TRIMESTRE',
 'REALIZADA',
 'REGION',
 'MAS_500',
 'AGLOMERADO',
 'PONDERA',
 'II1',
 'II2',
 'II5_1',
 'IX_TOT',
 'IX_MEN10',
 'IX_MAYEQ10',
 'ITF',
 'DECIFR',
 'GDECIFR',
 'ADECIFR',
 'IPCF',
 'DECCFR',
 'RDECCFR',
 'GDECCFR',
 'ADECCFR',
 'PONDIH',
 'VII1_1',
 'VII1_2',
 'VII2_1',
 'VII2_2',
 'VII2_3',
 'VII2_4']

In [229]:
#Armo una lista de las variables categoricas para hacer dummies. Son todas las variables menos las binarias y las numericas continuas. 
categorica24 = list(set(lista24) - set(nocategorica24))
len(categorica24)

60

In [230]:
for i in categorica24: 
    print(combined_24[i].value_counts())
    print("\n")

CH04
2    3442
1    3212
Name: count, dtype: int64


II4_3
2    4910
1    1742
0       2
Name: count, dtype: int64


V3
2    6584
1      44
9      26
Name: count, dtype: int64


IV8
1    6638
2      16
Name: count, dtype: int64


V14
2    5657
1     971
9      26
Name: count, dtype: int64


IV1
1    4728
2    1874
4      32
6      12
3       5
5       3
Name: count, dtype: int64


V16
2    3620
1    3008
9      26
Name: count, dtype: int64


CH14
2.0     4086
3.0      604
0.0      554
1.0      509
4.0      413
5.0      260
99.0     126
6.0       53
98.0      34
9.0        7
8.0        5
7.0        3
Name: count, dtype: int64


II7
1    4220
3    1186
2     524
6     435
4     158
8      80
9      18
7      16
5      15
0       2
Name: count, dtype: int64


V8
2    6436
1     192
9      26
Name: count, dtype: int64


V1
1    5894
2     734
9      26
Name: count, dtype: int64


II5
2    6172
0     420
1      62
Name: count, dtype: int64


CH13
1    3236
2    3227
0     173
9      18
Name

In [231]:
#Convierto los valores de las columnas categoricas a objetos para poder hacer dummies
for i in categorica24: 
    combined_24[i] = combined_24[i].astype('object')

In [232]:
dummies24 = pd.get_dummies(combined_24[categorica24])
combined_24_cat = pd.concat([combined_24, dummies24], axis = 1)

### combined_24_cat ----- incluye dummies Y las variables originales. 
combined_24 = combined_24_cat.drop(columns = categorica24)

### ANALISIS EXPLORATORIO: VARIABLES EXTRA, DESCRIPCION

In [233]:
#DEFINIR VARIABLES EXTRA
# Crear una nueva columna 'desocupado' con base en la condición ESTADO = 2
combined_04['desocupado'] = combined_04['ESTADO'] == 'Desocupado'
combined_24['desocupado'] = combined_24['ESTADO'] == 2

# Convertir valores booleanos a num
combined_04['desocupado'] = combined_04['desocupado'].map({True:1, False: 0})
combined_24['desocupado'] = combined_24['desocupado'].map({True:1, False: 0})


In [ ]:
#DEFINIR VARIABLES EXTRA para cada ano

# Verificar el resultado
#NSE - con ingreso. agrupar variables  IPCF
# bins = [0, 3000, 5000, 7000, 10000] 
# labels = ['Bajo', 'Medio-Bajo', 'Medio', 'Alto']
# combined_04['ingreso_agrupado'] = pd.cut(df['ingreso'], bins=bins, labels=labels, right=False
                                         
#subsidios V5, V12, V19_A (MENORES DE 10 AÑOS)

#proporcion del hogar que recibio educacion. CH10. 

In [ ]:
# df['ingreso_agrupado'] = pd.qcut(df['ingreso'], q=[0, 0.33, 0.66, 1.0], labels=['Menores 33%', '34%-66%', '67%-100%'])

In [ ]:
# #### VARIABLE DE EDUCACION POR HOGAR ###
# # Definir una función que mapee los valores de CH10 a categorías
# def map_educacion():
#     if combined_04['CH10'] == 1|combined_04['CH10']==2:
#         return 1
#     else:
#         return 0

# # Aplicar la función y crear la nueva columna 'educacion'
# combined_24['educacion'] = combined_24.apply(map_educacion)


In [ ]:

# # Agrupar por NRO_HOGAR y educacion (CH10)
# educacion_group = combined_24.groupby(['NRO_HOGAR', 'educacion']).size().reset_index(name='count')

# # Mostrar el resultado
# print(educacion_group)

In [38]:
df

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH04,CH06,CH13,CH15_COD,CH16_COD,ESTADO,...,IV4_Chapa de metal sin cubierta,IV4_Chapa de fibrocemento/plástico,IV4_Chapa de cartón,IV4_Caña/tabla/paja con barro/paja sola,IV4_N/S. Depto. de propiedad horizontal,CH09_Sí,CH09_No,CH09_Menor de 2 años,CH09_Ns./Nr.,desocupado
3,125169,1.0,1.0,Sí,Varón,72.0,No,34.0,NaN,Ocupado,...,False,False,False,False,False,True,False,False,False,0
4,125169,1.0,2.0,Sí,Mujer,66.0,No,86.0,NaN,Inactivo,...,False,False,False,False,False,True,False,False,False,0
5,125169,1.0,3.0,Sí,Mujer,41.0,No,86.0,NaN,Ocupado,...,False,False,False,False,False,True,False,False,False,0
6,125169,1.0,4.0,Sí,Varón,16.0,No,NaN,NaN,Inactivo,...,False,False,False,False,False,True,False,False,False,0
7,125169,1.0,5.0,Sí,Mujer,13.0,No,NaN,NaN,Inactivo,...,False,False,False,False,False,True,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7642,288171,1.0,3.0,0.0,Mujer,1.0,0.0,NaN,NaN,Menor de 10 años,...,True,False,False,False,False,False,False,True,False,0
7643,288405,1.0,1.0,Sí,Varón,51.0,Sí,30.0,NaN,Ocupado,...,False,False,False,False,False,True,False,False,False,0
7644,288405,1.0,2.0,Sí,Mujer,55.0,Sí,221.0,NaN,Ocupado,...,False,False,False,False,False,True,False,False,False,0
7645,288405,1.0,3.0,Sí,Mujer,12.0,No,NaN,NaN,Inactivo,...,False,False,False,False,False,True,False,False,False,0


In [41]:
#ANTES DE HACER ESTO HAY QUE ASEGURARNOS QUE ESTEN EN NUMEROS LAS CATEGORICAS

df = combined_04

# Calcular el número total de integrantes del hogar
total_integrantes = df.groupby('CODUSU')['NRO_HOGAR'].nunique()

# Calcular el número de integrantes que respondieron 1 o 2 en CH10
educados = df[['CH10_Sí, asiste','CH10_No asiste, pero asistió']].max(axis=1)#.groupby('NRO_HOGAR')['COMPONENTE'].nunique()

# Crear la variable prop_educado
df['prop_educado'] = df['NRO_HOGAR'].map(educados)/ df['NRO_HOGAR'].map(total_integrantes)

# Llenar los NaN con 0 (caso de hogares sin integrantes educados)
df['prop_educado'] = df['prop_educado'].fillna(0)

print(df['prop_educado'])

3       0
4       0
5       0
6       0
7       0
       ..
7642    0
7643    0
7644    0
7645    0
7646    0
Name: prop_educado, Length: 5131, dtype: int64


C:\Users\magda\AppData\Local\Temp\ipykernel_2880\2763209024.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['prop_educado'] = df['prop_educado'].fillna(0)


## Clasificacion y regularizacion

In [234]:
#FILTRAR PERSONAS Q NO RESPONDIERON ESTADO=0
#ESTADO: 0 = Entrevista individual no realizada (no respuesta al cuestionario individual)
    # 1 = Ocupado
    # 2 = Desocupado
    # 3 = Inactivo
    # 4 = Menor de 10 años

# largo_04 = len (combined_04)
# largo_24 = len (combined_24)

datos_04 = combined_04[combined_04['ESTADO'] != 0]
datos_24 = combined_24 [combined_24 ['ESTADO'] !=0]

# print (f'cant original 04',largo_04)
# print (f'cant nhueva 04', len(datos_04))

# print (f'cant original 24',largo_24)
# print (f'cant nhueva 24', len(datos_24))
# columnas_datos04 = datos_04.columns.tolist()
# columnas_datos04

#### Separación base Respondieron / no respondieron
Problema: En 2004 no hay gente que no haya respondido
- En 2024 hay 40

In [ ]:
#ACA HAY QUE DEFINIR LA BASE DE RESPONDIERON Y NO RESPONDIERON PARA CADA AÑO: 

# # DataFrame para las personas con ESTADO = 0
# norespondieron_04 = combined_04[combined_04['ESTADO'] == 0]
# norespondieron_24 = combined_24[combined_24['ESTADO'] == 0]

# # DataFrame para las personas con ESTADO distinto de 0
# respondieron_04 = combined_04[combined_04['ESTADO'] != 0]
# respondieron_24 = combined_24[combined_24['ESTADO'] != 0]

# # Imprimir el número de observaciones en cada DataFrame
# print(f"Total de personas que no respondieron (2004): {len(norespondieron_04)}")
# print(f"Total de personas que respondieron (2004) {len(respondieron_04)}")

# print(f"Total de personas que no respondieron (2024): {len(norespondieron_24)}")
# print(f"Total de personas que respondieron (2024) {len(respondieron_24)}")

Total de personas que no respondieron (2004): 0
Total de personas que respondieron (2004) 7627
Total de personas que no respondieron (2024): 40
Total de personas que respondieron (2024) 6614


In [235]:
# Se separa X e y a partir de la selección de columnas numericas
#ESTO HAY QUE UPDATEARLO CON LAS BASES FINALES
X_2004 = datos_04.drop(columns=['desocupado'])
y_2004 = datos_04['desocupado']
X_2024 = datos_24.drop(columns=['desocupado'])
y_2024 = datos_24['desocupado']

In [236]:
#Partir la base train test para cada ano. 
#Semilla 101
#agregar columna de unos (1)
x_train_24, x_test_24, y_train_24, y_test_24 = train_test_split(X_2024, y_2024, test_size=0.3, random_state=101)
x_train_04, x_test_04, y_train_04, y_test_04 = train_test_split(X_2004, y_2004, test_size=0.3, random_state=101)

x_train_04 = sm.add_constant(x_train_04) #AGREGO COLUMNA DE UNOS
x_train_24 = sm.add_constant(x_train_24)

In [61]:
tipos_de_datos = x_train_04.dtypes 
print(tipos_de_datos)

CODUSU                    object
NRO_HOGAR                float64
COMPONENTE              category
H15                     category
CH04                    category
                          ...   
CH09_Sí                     bool
CH09_No                     bool
CH09_Menor de 2 años        bool
CH09_Ns./Nr.                bool
prop_educado               int64
Length: 249, dtype: object


In [237]:
x_train_04.columns.to_list()

['CODUSU',
 'NRO_HOGAR',
 'COMPONENTE',
 'H15',
 'CH04',
 'CH06',
 'CH13',
 'ESTADO',
 'P47T',
 'DECINDR',
 'RDECINDR',
 'GDECINDR',
 'ADECINDR',
 'V2_M',
 'V3_M',
 'V4_M',
 'V5_M',
 'V8_M',
 'V9_M',
 'V10_M',
 'V11_M',
 'V12_M',
 'V18_M',
 'V19_AM',
 'V21_M',
 'T_VI',
 'REALIZADA',
 'ANO4',
 'TRIMESTRE',
 'REGION',
 'MAS_500',
 'AGLOMERADO',
 'PONDERA',
 'IV2',
 'IV5',
 'IV8',
 'IV12_1',
 'IV12_2',
 'IV12_3',
 'II1',
 'II2',
 'II4_1',
 'II4_2',
 'II4_3',
 'II5',
 'II5_1',
 'V1',
 'V2',
 'V21',
 'V22',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19_A',
 'V19_B',
 'IX_TOT',
 'IX_MAYEQ10',
 'ITF',
 'DECIFR',
 'RDECIFR',
 'GDECIFR',
 'ADECIFR',
 'IPCF',
 'DECCFR',
 'RDECCFR',
 'GDECCFR',
 'ADECCFR',
 'VII1_1',
 'VII1_2',
 'VII2_1',
 'VII2_2',
 'VII2_3',
 'VII2_4',
 'IV1_Casa',
 'IV1_Departamento',
 'IV1_Pieza de inquilinato',
 'IV1_Pieza en hotel/pensión',
 'IV1_Local no construido para habitación',
 'IV1_Otro

In [ ]:
# Identificar las columnas float e int
columnas_a_estandarizar_04 = x_train_04.select_dtypes(include=['int64', 'float64']).columns
columnas_a_estandarizar_24 = x_train_24.select_dtypes(include=['int64', 'float64']).columns

# Inicializar el estandarizador
sc04 = StandardScaler()
sc24 = StandardScaler()

# Estandarizamos las observaciones de entrenamiento
x_train_04_transformed = x_train_04.copy()
x_train_04_transformed[columnas_a_estandarizar_04] = sc04.fit_transform(x_train_04[columnas_a_estandarizar_04])

x_train_24_transformed = x_train_24.copy()
x_train_24_transformed[columnas_a_estandarizar_24] = sc24.fit_transform(x_train_24[columnas_a_estandarizar_24])

# Estandarizamos las observaciones de test usando el estandarizador ya ajustado
x_test_04_transformed = x_test_04.copy()
x_test_04_transformed[columnas_a_estandarizar_04] = sc04.transform(x_test_04[columnas_a_estandarizar_04])

x_test_24_transformed = x_test_24.copy()
x_test_24_transformed[columnas_a_estandarizar_24] = sc24.transform(x_test_24[columnas_a_estandarizar_24])

# Estadisticas luego de estandarizar
print(x_train_04_transformed.describe().T)
print(x_train_24_transformed.describe().T)


c:\Users\magda\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\magda\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\magda\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


             count          mean       std       min       25%       50%  \
NRO_HOGAR   3591.0  2.671213e-17  1.000139 -0.162687 -0.162687 -0.162687   
P47T        3591.0  2.572279e-17  1.000139 -0.630567 -0.630567 -0.630567   
DECINDR     3591.0  3.759485e-17  1.000139 -0.681451 -0.681451 -0.681451   
RDECINDR    3591.0 -5.540294e-17  1.000139 -0.669856 -0.669856 -0.669856   
GDECINDR    3591.0 -3.561618e-17  1.000139 -0.675831 -0.675831 -0.675831   
ADECINDR    3591.0 -7.914706e-17  1.000139 -0.668473 -0.668473 -0.668473   
V2_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V3_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V4_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V5_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V8_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V9_M        3591.0  0.000000e+00  0.000000  0.000000  0.000000  0.000000   
V10_M       

In [244]:
x_train_04_transformed['H15'].value_counts()

H15
Sí     2877
0.0     706
No        8
Name: count, dtype: int64

In [242]:
#Regresion logistica con penalidad LASSO (todos estos pasos se hacen para cada ano)
#### Regresion logistica para 2004
alpha = 1
log04_lasso = LogisticRegression(penalty='l1', C=(1/alpha), solver='liblinear').fit(x_train_04, y_train_04)
y_test_pred_score_04_lasso = log04_lasso.predict_proba(x_train_04)[:,1]
y_pred_log04_lasso = log04_lasso.predict(x_test_04)

#### Regresion logistica para 2024
log24_lasso = LogisticRegression(penalty='l1', C=(1/alpha), solver='liblinear').fit(x_train_24, y_train_24)
y_test_pred_score_24_lasso = log24_lasso.predict_proba(x_train_24)[:,1]
y_pred_log24_lasso = log24_lasso.predict(x_test_24)

ValueError: could not convert string to float: 'S'

In [ ]:
#### Medidas de precision Regresion logistica LASSO para 2004
#Matriz de confusion
conflog04_lasso = confusion_matrix(y_test_04, y_pred_log04_lasso) 
print("Matriz de confusion (Reg Log LASSO 2004)\n", conflog04_lasso)

#AUC
auc_log04_lasso = roc_auc_score(y_test_04, y_pred_log04_lasso)
print('\nAUC Reg Log (2004): %.4f' %auc_log04_lasso)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_04, y_pred_log04_lasso)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log04_lasso, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_l04 = mean_squared_error(y_test_04, y_pred_log04_lasso)
print("Error cuadrático medio:", ecm_a1_l04) 

#Accuracy
accuracy_log04_lasso = accuracy_score(y_test_04, y_pred_log04_lasso)
print("La accuracy del modelo (Reg Log LASSO 2004) es: %.4f" %accuracy_log04_lasso)

In [ ]:
#### Medidas de precision Regresion logistica LASSO para 2024
#Matriz de confusion
conflog24_lasso = confusion_matrix(y_test_24, y_pred_log24_lasso) 
print("Matriz de confusion (Reg Log LASSO 2024)\n", conflog24_lasso)

#AUC
auc_log24_lasso = roc_auc_score(y_test_24, y_pred_log24_lasso)
print('\nAUC Reg Log LASSO (2024): %.4f' %auc_log24_lasso)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_24, y_pred_log24_lasso)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log24_lasso, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_l24 = mean_squared_error(y_test_24, y_pred_log24_lasso)
print("Error cuadrático medio:", ecm_a1_l24) 

#Accuracy
accuracy_log24_lasso = accuracy_score(y_test_24, y_pred_log24_lasso)
print("La accuracy del modelo (Reg Log LASSO 2024) es: %.4f" %accuracy_log24_lasso)

In [ ]:
#Regresion logistica con penalidad ridge (todos estos pasos se hacen para cada ano)
#### Regresion logistica para 2004
alpha = 1
log04_ridge = LogisticRegression(penalty='l2', C=(1/alpha), solver='liblinear').fit(x_train_04, y_train_04)
y_test_pred_score_04_ridge = log04_ridge.predict_proba(x_train_04)[:,1]
y_pred_log04_ridge = log04_ridge.predict(x_test_04)

#### Regresion logistica para 2024
log24_ridge = LogisticRegression(penalty='l2', C=(1/alpha), solver='liblinear').fit(x_train_24, y_train_24)
y_test_pred_score_24_ridge = log24_ridge.predict_proba(x_train_24)[:,1]
y_pred_log24_ridge = log24_ridge.predict(x_test_24)

In [ ]:
#### Medidas de precision Regresion logistica ridge para 2004
#Matriz de confusion
conflog04_ridge = confusion_matrix(y_test_04, y_pred_log04_ridge) 
print("Matriz de confusion (Reg Log ridge 2004)\n", conflog04_ridge)

#AUC
auc_log04_ridge = roc_auc_score(y_test_04, y_pred_log04_ridge)
print('\nAUC Reg Log ridge (2004): %.4f' %auc_log04_ridge)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_04, y_pred_log04_ridge)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log04_ridge, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_r04 = mean_squared_error(y_test_04, y_pred_log04_ridge)
print("Error cuadrático medio:", ecm_a1_r04)  

#Accuracy
accuracy_log04_ridge = accuracy_score(y_test_04, y_pred_log04_ridge)
print("La accuracy del modelo (Reg Log ridge 2004) es: %.4f" %accuracy_log04_ridge)

In [ ]:
#### Medidas de precision Regresion logistica ridge para 2024
#Matriz de confusion
conflog24_ridge = confusion_matrix(y_test_24, y_pred_log24_ridge) 
print("Matriz de confusion (Reg Log ridge 2024)\n", conflog04_ridge)

#AUC
auc_log24_ridge = roc_auc_score(y_test_24, y_pred_log24_ridge)
print('\nAUC Reg Log ridge (2024): %.4f' %auc_log24_ridge)

#Curva ROC
fpr, tpr, thresholds = roc_curve(y_test_24, y_pred_log24_ridge)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_log24_ridge, estimator_name='Reg Log')
display.plot()  
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.show() 

#MSE
ecm_a1_r24 = mean_squared_error(y_test_24, y_pred_log24_ridge)
print("Error cuadrático medio:", ecm_a1_r24) 

#Accuracy
accuracy_log24_ridge = accuracy_score(y_test_24, y_pred_log24_ridge)
print("La accuracy del modelo (Reg Log ridge 2024) es: %.4f" %accuracy_log24_ridge)

In [ ]:
# Cross validation para elegir el alpha para los metodos de regularizacion
from sklearn.linear_model import LogisticRegressionCV

#LASSO 2004
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_lasso_cv_04 = LogisticRegressionCV(Cs=10, cv=10, penalty='l1', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_lasso_cv_04.fit(x_train_04, y_train_04)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_lasso_cv_04.C_[0])

print("El mejor valor de alpha:", (1/log_lasso_cv_04.C_[0]))

# # Hacer predicciones (opcional)
# y_pred_ = log_lasso_cv_04.predict(x_test_04)

#LASSO 2024
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_lasso_cv_24 = LogisticRegressionCV(Cs=10, cv=10, penalty='l1', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_lasso_cv_24.fit(x_train_24, y_train_24)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_lasso_cv_24.C_[0])

print("El mejor valor de alpha:", (1/log_lasso_cv_24.C_[0]))

In [ ]:
#Ridge 2004
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_ridge_cv_04 = LogisticRegressionCV(Cs=10, cv=10, penalty='l2', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_ridge_cv_04.fit(x_train_04, y_train_04)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_ridge_cv_04.C_[0])

print("El mejor valor de alpha:", (1/log_ridge_cv_04.C_[0]))

# # Hacer predicciones (opcional)
# y_pred_ = log_lasso_cv_04.predict(x_test_04)

#Ridge 2024
# Crear el modelo de regresión logística con validación cruzada y penalización L1
log_ridge_cv_24 = LogisticRegressionCV(Cs=10, cv=10, penalty='l2', solver='saga', scoring='accuracy', refit=True)

# Entrenar el modelo
log_ridge_cv_24.fit(x_train_24, y_train_24)

# Imprimir el mejor valor de C (inverso de alpha)
print("Mejor valor de C:", log_ridge_cv_24.C_[0])

print("El mejor valor de alpha:", (1/log_ridge_cv_24.C_[0]))

In [ ]:
#boxplot usando seaborn: distribucion del error de prediccion para cada alpha

#lineplot para lasso: promedio de la proporcion de variables ignoradas por el modelo 

In [ ]:
#MODELOS LASSO OPTIMOS

#2004

#2024

In [ ]:
#MODELOS RIDGE OPTIMOS

#2004

#2024

In [ ]:
#comparar resultados lasso y ridge 2004 vs 2024